In [1]:
import warnings
import pandas as pd
import arviz as az

import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)
#from libs.model import HGPforecaster
from libs.metrics import calculate_metrics, metrics_to_table, metrics_to_latex
from libs.pre_processing import generate_groups_data_flat
from libs.visual_analysis import visualize_fit, visualize_predict
from libs.model import HGPforecaster
import numpy as np
az.style.use('arviz-darkgrid')
warnings.filterwarnings('ignore')

import theano
theano.config.compute_test_value='raise'
#theano.config.compute_test_value = "warn"

In [2]:
prison = pd.read_csv('../data/prisonLF.csv', sep=",")
prison = prison.drop('Unnamed: 0', axis =1)
prison['t'] = prison['t'].astype('datetime64[ns]')

In [3]:
prison_pivot = prison.pivot(index='t',columns=['state', 'gender', 'legal'], values='count')

In [4]:
groups_input = {
    'state': [0],
    'gender': [1],
    'legal': [2]
}

In [5]:
groups = generate_groups_data_flat(y = prison_pivot, 
                               groups_input = groups_input, 
                               seasonality=4, 
                               h=8)

Number of groups: 3
	state: 8
	gender: 2
	legal: 2
Total number of series: 32
Number of points per series for train: 40
Total number of points: 48
Seasonality: 4
Forecast horizon: 8


# Testing with minibatch

In [6]:
m = HGPforecaster(groups_data=groups,
                  n_iterations=100000,
                  log_lin_mean=True)#,
                  #minibatch=[30,20])

In [ ]:
m.fit_vi()

Fitting model...


In [ ]:
m.predict()

In [ ]:
visualize_predict(groups, m.pred_samples_predict['y_pred_new'], 8)

In [ ]:
results = calculate_metrics(m.pred_samples_predict['y_pred_new'], groups)

In [ ]:
metrics_to_table(groups, results)